In [1]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

from sklearn.metrics import r2_score
from sklearn.metrics import explained_variance_score
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import median_absolute_error
from sklearn.metrics import mean_squared_error

## Importing data from Excel Files

In [2]:
path = "PET_PRI_SPT_S1_M.xls"
test_path = "testResults.csv"

#Array of Dataframes input
data = []

for i in range(1, 8):
    data.append(pd.read_excel(path, sheet_name = i, skiprows = [0,1]))

In [3]:
#result storing average value and price change per year
resultdf = []

#result storing average value and price change per year
resultdf_monthly = []

#Types of Data Available
types = ['Crude Oil', 'Conventional Gasoline', 'RBOB Regular Gasoline', 'Heating Oil', 'Diesel Fuel', 'Kerosene Type Jet Fuel', 'Propane']

## Analysis

In [4]:
def analyze(data2, ind):
    monthList = []
    yearList = []
    monthDiff = []
    nextData = []
    row_iterator = data2.iterrows()
    
    str_cols = data2.columns[data2.dtypes==object]
    data2[str_cols] = data2[str_cols].fillna('.')
    data2.fillna(0,inplace=True)
    
    for index, row in row_iterator:
        current_date = row['Date'] 
        monthList.append(current_date.month)
        yearList.append(current_date.year)
        if(index == len(data2)-1):
            nextData.append(data2.iloc[index][1])
            monthDiff.append(0)
        else:
            nextData.append(data2.iloc[index+1][1])
            monthDiff.append(data2.iloc[index+1][1] - row[1])
    
    data2['Month'] = monthList
    data2['Year'] = yearList
    data2['Monthly Change in Price'] = monthDiff     

    map_month = dict.fromkeys(data2.Month.unique())
    
    for month in data2.Month.unique():
        df1 = data2.loc[data2['Month'] == month]
        print('*******')
        print('For month', month)
        print(df1.iloc[:,1])
        mean_mnth = np.asarray(df1.iloc[:,1]).mean()
        print('mean:',mean_mnth )
        map_month[month] = mean_mnth
    
    yearls_m = list(map_month.keys())
    print(map_month)

    meanls_m = list(map_month.values())
    
    res = pd.DataFrame(np.column_stack([yearls_m, meanls_m]), 
                               columns=['Month', 'Mean Price of '+str(types[ind])])

    resultdf_monthly.append(res) 

    map_ = dict.fromkeys(data2.Year.unique())
    
    
    for yr in data2.Year.unique():
        df = data2.loc[data2['Year'] == yr]
        mean = np.asarray(df.iloc[:,1]).mean()
        map_[yr] = mean
    
    yearls = list(map_.keys())

    meanls = list(map_.values())
    
    result = pd.DataFrame(np.column_stack([yearls, meanls]), 
                               columns=['Year', 'Mean Price of '+str(types[ind])])
    
    yearDiff = []
    for index, row in result.iterrows():
        if(index == len(result)-1):
            yearDiff.append(0)
        else:
            yearDiff.append(result.iloc[index+1][1] - row[1])
    
    result['Yearly Change in Price'] = yearDiff 
    
    resultdf.append(result) 

## Training Linear Regression, Random Forest, and Decision Tree Models

In [6]:
def trainModels(file, imp_attr):
    #split into train and test
    train, test =  train_test_split(file, test_size = 0.2)

    #Removing the target/predictor from the train data
    targ = train[list(target_col)]
    
    ### Random Forest Model
    rand_forest_model =  RandomForestRegressor(n_estimators = 1000 , max_features = 2, oob_score = True ,  random_state = 115)
    rand_forest_model.fit(train[list(imp_attr)],targ)
    
    ### Decision Tree Model
    decision_tree_model = DecisionTreeRegressor(max_depth=4)
    decision_tree_model.fit(train[list(imp_attr)],targ)    
   
    ### Linear Regression Model
    linear_model = LinearRegression()
    linear_model.fit(train[list(imp_attr)], targ)
    
    return rand_forest_model, decision_tree_model, linear_model, test

## Evaluating the models

In [7]:
def  testNEvalModels(test, rf_model, dt_model, lm_model, imp_attr):
   
    print('\n Evaluation Staistics:')
    ### Evaluating Random Forest
    print("\n ***Random Forest Regressor***")
    #Evaluation metric: r square
    r2 = r2_score(test[list(target_col)] , rf_model.predict(test[list(imp_attr)]))
    print("R-Square Value:", r2)
    
    # extracting the test target values and convert to float
    true_vals = test[list(target_col)].values
    true_vals_flt = true_vals.astype(np.float)
    
    prediction = rf_model.predict(test[list(imp_attr)])
    
    #reshaping the array is required to convert it into numpy array
    aa = prediction.reshape(-1,1)
    
    mean_squared_error(true_vals_flt, prediction)
    
    mse = np.mean((true_vals_flt - aa)**2)
    print("Mean Squared Error", mse)
    
    print("Explained Variance Score", explained_variance_score(true_vals_flt, prediction))     
    print("Mean Absolute Error", mean_absolute_error(true_vals_flt, prediction))
    print("Median Absolute Error", median_absolute_error(true_vals_flt, prediction))    
    
    ### Evaluating Decision tree
    print("\n ***Decision Tree Regressor***")
    y_2 = dt_model.predict(test[list(imp_attr)])
    r2_dt = r2_score(true_vals_flt , y_2)
    print("R-Square Value:", r2_dt)
    
    print("Mean Squared Error", mean_squared_error(true_vals_flt, y_2))  
    
    print("Explained Variance Score", explained_variance_score(true_vals_flt, y_2))     
    print("Mean Absolute Error", mean_absolute_error(true_vals_flt, y_2))
    print("Median Absolute Error", median_absolute_error(true_vals_flt, y_2))    
    
    
    ### Evaluating Linear Model
    print("\n ***Linear Regression Model***")
    pred_lm = lm_model.predict(test[list(imp_attr)])

    r2_lm = r2_score(true_vals_flt, pred_lm)
    
    print("R-Square Value:", r2_lm)
    
    print("Mean Squared Error", mean_squared_error(true_vals_flt, pred_lm))  
    
    print("Explained Variance Score", explained_variance_score(true_vals_flt, pred_lm))     
    print("Mean Absolute Error", mean_absolute_error(true_vals_flt, pred_lm))
    print("Median Absolute Error", median_absolute_error(true_vals_flt, pred_lm))   

## Prediction

In [9]:
def applyModel(model, test):
    #Apply selected model to test data
    pred = model.predict(test[list(imp_attr)])

    #save predicted into target column in test
    test_dtm['Predicted Price'] = pred
    
    #save df to file
    test_dtm.to_csv(test_path)

In [10]:
writer = pd.ExcelWriter('output.xlsx')
writer_m = pd.ExcelWriter('output_m.xlsx')

## Analyzing data

In [11]:
for i in range(len(data)):
    analyze(data[i], i)

*******
For month 1
0       22.93
12      18.65
24      17.13
36      18.02
48      22.86
60      25.23
72      18.79
84      19.03
96      15.03
108     18.04
120     18.86
132     25.13
144     16.72
156     12.52
168     27.26
180     29.59
192     19.72
204     32.95
216     34.31
228     46.84
240     65.49
252     54.51
264     92.97
276     41.71
288     78.33
300     89.17
312    100.27
324     94.76
336     94.62
348     47.22
360     31.68
372     52.50
384     63.70
396     51.38
408     57.52
420     52.00
Name: Cushing, OK WTI Spot Price FOB (Dollars per Barrel), dtype: float64
mean: 42.42888888888889
*******
For month 2
1       15.46
13      17.75
25      16.80
37      17.94
49      22.11
61      20.48
73      19.01
85      20.09
97      14.78
109     18.57
121     19.09
133     22.18
145     16.06
157     12.01
169     29.37
181     29.61
193     20.72
205     35.83
217     34.69
229     48.15
241     61.63
253     59.28
265     95.39
277     39.09
289     76.39
301     

*******
For month 6
0      1.072
12     1.457
24     1.745
36     2.401
48     2.354
60     3.694
72     2.085
84     2.255
96     2.861
108    2.626
120    3.021
132    3.116
144    2.187
156    1.642
168    1.619
180    2.136
192    1.851
204    1.301
216    2.311
Name: Los Angeles Reformulated RBOB Regular Gasoline Spot Price (Dollars per Gallon), dtype: float64
mean: 2.1965263157894737
*******
For month 7
1      0.965
13     1.467
25     1.882
37     2.488
49     2.311
61     3.286
73     1.913
85     2.281
97     2.948
109    2.819
121    3.047
133    2.908
145    2.663
157    1.412
169    1.669
181    2.140
193    1.919
205    1.328
217    2.361
Name: Los Angeles Reformulated RBOB Regular Gasoline Spot Price (Dollars per Gallon), dtype: float64
mean: 2.2003684210526315
*******
For month 8
2      1.315
14     1.388
26     2.142
38     2.190
50     2.049
62     3.072
74     2.113
86     2.178
98     2.839
110    3.129
122    2.825
134    2.808
146    2.075
158    1.388
170    1.807

*******
For month 4
0      0.000
12     0.000
24     0.000
36     0.000
48     0.000
60     0.000
72     0.000
84     0.000
96     0.000
108    0.000
120    0.000
132    2.053
144    3.443
156    1.449
168    2.284
180    3.271
192    3.242
204    2.900
216    2.958
228    1.843
240    1.247
252    1.586
264    2.079
276    2.062
288    0.880
300    1.861
Name: New York Harbor Ultra-Low Sulfur No 2 Diesel Spot Price (Dollars per Gallon), dtype: float64
mean: 1.275307692307692
*******
For month 5
1      0.000
13     0.000
25     0.000
37     0.000
49     0.000
61     0.000
73     0.000
85     0.000
97     0.000
109    0.000
121    0.000
133    2.044
145    3.821
157    1.520
169    2.136
181    3.035
193    2.987
205    2.888
217    2.936
229    1.965
241    1.423
253    1.514
265    2.230
277    2.031
289    0.887
301    2.024
Name: New York Harbor Ultra-Low Sulfur No 2 Diesel Spot Price (Dollars per Gallon), dtype: float64
mean: 1.2861923076923079
*******
For month 6
2      0.000
14  

*******
For month 6
0      0.344
12     0.328
24     0.288
36     0.318
48     0.348
60     0.344
72     0.245
84     0.309
96     0.555
108    0.431
120    0.375
132    0.559
144    0.670
156    0.818
168    1.097
180    1.139
192    1.813
204    0.846
216    1.037
228    1.520
240    0.788
252    0.863
264    1.046
276    0.369
288    0.507
300    0.590
312    0.880
324    0.449
336    0.496
348    0.965
Name: Mont Belvieu, TX Propane Spot Price FOB (Dollars per Gallon), dtype: float64
mean: 0.6779000000000001
*******
For month 7
1      0.342
13     0.314
25     0.292
37     0.308
49     0.356
61     0.348
73     0.246
85     0.372
97     0.551
109    0.390
121    0.372
133    0.530
145    0.741
157    0.845
169    1.165
181    1.190
193    1.862
205    0.752
217    1.010
229    1.528
241    0.874
253    0.920
265    1.036
277    0.409
289    0.478
301    0.647
313    0.938
325    0.487
337    0.491
349    1.090
Name: Mont Belvieu, TX Propane Spot Price FOB (Dollars per Gallon), dtyp

## Yearly Analysis

In [13]:
for i in range(len(resultdf)):
    resultdf[i].to_excel(writer,'data'+str(i))
writer.save()

## Monthly Analysis

In [14]:
for i in range(len(resultdf_monthly)):
    resultdf_monthly[i].to_excel(writer_m,'data'+str(i))
writer_m.save()

## Training, Evaluation, and Predictions

In [19]:
#Crude oil data being taken as input
train_data = data[0]
target_col = ["Cushing, OK WTI Spot Price FOB (Dollars per Barrel)"]

#features selected on which the model would be based on
imp_attr = ['Month','Year']

# Training different models and choosing the best one for prediction
rf,dt,lm,test = trainModels(train_data, imp_attr)
testNEvalModels(test, rf, dt, lm, imp_attr)

test_dtm={}
test_dtm['Month']=[5,6,7,8,9,10]
test_dtm['Year']=[2017, 2017, 2017, 2017, 2017, 2017]
test_dtm = pd.DataFrame.from_dict(test_dtm)

applyModel(rf, test_dtm)

print("Yearly Change in Price is stored in file named: output.xlsx")
print("Monthly Change in Price is stored in file named: output_m.xlsx")
print("The resultant Predicted Values are stored in a File named: testResults.csv")

<ipython-input-6-5af248d17b67>:10: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  rand_forest_model.fit(train[list(imp_attr)],targ)



 Evaluation Staistics:

 ***Random Forest Regressor***
R-Square Value: 0.9504364930071154
Mean Squared Error 31.582863045870596
Explained Variance Score 0.9504684930587266
Mean Absolute Error 3.156366860465094
Median Absolute Error 1.3701950000001304

 ***Decision Tree Regressor***
R-Square Value: 0.8770844506933922
Mean Squared Error 78.32425902621985
Explained Variance Score 0.8771087970258189
Mean Absolute Error 5.111977725541833
Median Absolute Error 2.650370370370368

 ***Linear Regression Model***
R-Square Value: 0.5100212789772907
Mean Squared Error 312.2242912244422
Explained Variance Score 0.5192870535537656
Mean Absolute Error 13.51834321700859
Median Absolute Error 10.207013071034435
Yearly Change in Price is stored in file named: output.xlsx
Monthly Change in Price is stored in file named: output_m.xlsx
The resultant Predicted Values are stored in a File named: testResults.csv


## Saving the best performing model

In [24]:
import joblib
joblib.dump(rf, "rf_model.joblib")

['rf_model.joblib']

In [35]:
data[0]

,Date,"Cushing, OK WTI Spot Price FOB (Dollars per Barrel)",Europe Brent Spot Price FOB (Dollars per Barrel),Month,Year,Monthly Change in Price
0,1986-01-15,22.93,0.00,1,1986,-7.47
1,1986-02-15,15.46,0.00,2,1986,-2.85
2,1986-03-15,12.61,0.00,3,1986,0.23
3,1986-04-15,12.84,0.00,4,1986,2.54
4,1986-05-15,15.38,0.00,5,1986,-1.95
...,...,...,...,...,...,...
425,2021-06-15,71.38,73.16,6,2021,1.11
426,2021-07-15,72.49,75.17,7,2021,-4.76
427,2021-08-15,67.73,70.75,8,2021,3.92
428,2021-09-15,71.65,74.49,9,2021,9.83


In [30]:
loaded_rf.predict([[7, 1988]])

array([16.61751])

In [41]:
loaded_rf.predict(test[list(imp_attr)])

array([ 16.61751,  19.04012,  19.00672,  28.64897,  49.60091,  45.95745,
       103.66525,  27.83916,  31.18393,  62.70822,  17.50572,  48.96782,
        75.77354,  19.58468,  56.67869,  64.81093,  43.12546,  19.43424,
        49.55296,  20.20399,  94.02067,  43.83015,  22.11997,  22.1559 ,
        15.16555,  93.48119,  20.49898,  98.45482,  15.18118,  21.17436,
        24.19627,  20.9094 ,  18.69905,  65.44522,  19.0494 ,  28.99009,
        44.32986,  97.02473,  18.58065,  21.6644 ,  17.81918,  32.04062,
        38.99651,  19.15662,  77.64675,  55.32809,  20.72343,  18.03406,
        17.29492,  19.06208,  91.04354,  31.71963,  22.21312,  52.77015,
        21.51539,  51.48736,  20.69361,  70.86613,  29.13927,  66.13713,
        43.46133,  68.55012,  22.27695,  40.6901 ,  58.93973,  51.05757,
        17.8329 ,  89.00234,  28.13489,  15.81129,  45.90141,  35.38998,
        61.16036,  97.15169,  20.84238,  40.13965,  71.05708,  20.43529,
        19.58447,  63.24635,  18.74066,  17.86777, 